# Import Library

This code is for importing library what I need.

In [1]:
!pip install numpy
!pip install pandas==1.5.3
!pip install nltk
!pip install beautifulsoup4
!pip install mpld3
!pip install -U scikit-learn
!pip install joblib
!pip install gensim
!pip install pyLDAvis==3.4.0

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 53.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.5
    Not uninstalling pandas at /shared-libs/python3.9/py/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'pandas'. No files were found to uninstall.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.2/201.2 KB 32.2 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 76.3 MB/s eta 0:00:00
 

In [2]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from bs4 import BeautifulSoup
import re
import os 
import codecs
from sklearn import feature_extraction
import mpld3
from glob import glob 
import json

import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-05 08:42:14.530273: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 08:42:14.676075: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot ope

# Document Clustering

### Stopwords, stemming, and tokenizing

This section is focused on defining some functions to manipulate the titles/bodies. First, I load NLTK's list of English stop words. Stop words are words like "a", "the", or "in" which don't convey significant meaning. I'm sure there are much better explanations of this out there.

In [3]:
# load nltk's English stopwords as variable called 'stopwords'
new_stopwords = ["use", "abl", "would", "like", "made", "happen", "id", "n\'t", "expect", "1", "useline", "arn", "aw", "int2byt", "py35", "python", "kw", "def", "six", "six.mov", "\'s", "ns", "-r", "int2byte_alt", "python3", "six.add_metaclass", "setup.pi", "import_modul", "r", "x", "__name__", "0m", "1m" ,"\'m", "'m", "flask", "pkgutil.get_load", "ani", "-g", "_configtest.c", "unknown._", "f", "\'d", "|sin", "|cos", "h", "numpy.float64", "|squar", "|absolut", "|sqrt", "|reciproc", "b", "-fwrapv", "defin", "x86_64-linux-gnu-gcc", "-fstack-protector-strong", "numpi", "µs", "ms", "//projects.scipy.org/numpy/ticket/709", "v", "datetime.datetime.now", "side=\'left", "bench_avx", "v_shape", "np.searchsort", "__init__.pi", "array_lik", "ss", "cdk", "aws-cdk", "kwarg", "name", "iam", "aws-cdk/cor", "aws-cdk/cfnspec", "callabl", "arg", "l", "0x485b9e", "0x0", "cjw296", "numpy.isnan", "xxx", "\\n", "__init__"]
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(new_stopwords)
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Next I import the Snowball Stemmer which is actually part of NLTK. Stemming is just the process of breaking a word down into its root.

In [4]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

Below I define two functions: 
tokenize_and_stem: tokenizes (splits the titles into a list of its respective words (or tokens) and also stems each token tokenize_only: tokenizes the titles only I use both these functions to create a dictionary which becomes important in case I want to use stems for an algorithm, but later convert stems back to their full words for presentation purposes. Guess what, I do want to do that!

In [5]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

### Latent Dirichlet Allocation

For my implementaiton of LDA, I use the Gensim package. I'm going to preprocess the titles a bit differently here, and first I define a function to remove any proper noun.

In [6]:
#strip any proper names from a text...unfortunately right now this is yanking the first word from a sentence too.
import string
def strip_proppers(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()

Here I run the actual text processing (removing of proper nouns, tokenization, removal of stop words)

In [7]:
#strip any proper nouns (NNP) or plural proper nouns (NNPS) from a text
from nltk.tag import pos_tag

def strip_proppers_POS(text):
    tagged = pos_tag(text.split()) #use NLTK's part of speech tagger
    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
    return non_propernouns

Here I run the actual text processing (removing of proper nouns, tokenization, removal of stop words)

In [8]:
#Latent Dirichlet Allocation implementation with Gensim

from gensim import corpora, models, similarities 

## Title and Body Newcomers Scenario

### Stopwords, stemming, and tokenizing

Access, split and compute data from newcomers body and title txt files.

In [9]:
#import data title
titles = open('data_title_newcomers.txt').read().split('\BREAKHERE')

#ensures that only the first 1825 are read in
titles = titles[1:1826]

#import data body
bodies = open('data_body_newcomers.txt').read().split('\BREAKHERE')

#ensures that only the first 1825 are read in
bodies = bodies[1:1826]

print(str(len(titles)) + ' titles')
print(str(len(bodies)) + ' bodies')

8 titles
8 bodies


Enter newcomers body and title data into the list

In [10]:
# generates index for each item
list_titles = []

for text in titles:
    text = BeautifulSoup(text, 'html.parser').getText()
    #strips html formatting and converts to unicode
    list_titles.append(text)

titles = list_titles

# generates index for each item
list_bodies = []

for text in bodies:
    text = BeautifulSoup(text, 'html.parser').getText()
    #strips html formatting and converts to unicode
    list_bodies.append(text)

bodies = list_bodies

bodies_titles = []

for i in range(len(bodies)):
    item = bodies[i] + titles[i]
    bodies_titles.append(item)

Check newcomers body and title data.

In [11]:
pd.DataFrame(bodies_titles)

,0
0,improve code quality\n\n\nSTY: unify imports i...
1,The signature is\n```python\nnp.searchsorted(a...
2,The Type checker inspection in PyCharm (2020.1...
3,I want to test the performance optimization of...
4,The error message produced by setting value of...
5,The docstring of the first argument of loadtxt...
6,_Original ticket http://projects.scipy.org/num...
7,


Below I use my stemming/tokenizing and tokenizing functions to iterate over the list of bodies and titles to create two vocabularies: one stemmed and one only tokenized.

In [12]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in bodies_titles:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

Using these two lists, I create a pandas DataFrame with the stemmed vocabulary as the index and the tokenized words as the column. The benefit of this is it provides an efficient way to look up a stem and return a full token. The downside here is that stems to tokens are one to many: the stem 'run' could be associated with 'ran', 'runs', 'running', etc. For my purposes this is fine--I'm perfectly happy returning the first token associated with the stem I need to look up.

In [13]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

Check newcomers bodies and titles data vocab frame.

In [14]:
vocab_frame

,words
improv,improve
code,code
qualiti,quality
sti,sty
unifi,unify
...,...
should,should
not,not
rais,raise
typeerror,typeerrors


Check the shape of items in vocab_frame

In [15]:
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 529 items in vocab_frame


### Latent Dirichlet Allocation

Here I run the actual text processing (removing of proper nouns, tokenization, removal of stop words)

In [16]:
#Latent Dirichlet Allocation implementation with Gensim

#remove proper names
preprocess = [strip_proppers(doc) for doc in bodies_titles]

%time tokenized_text = [tokenize_and_stem(text) for text in preprocess]

%time texts = [[word for word in text if word not in stopwords] for text in tokenized_text]

CPU times: user 5.73 ms, sys: 0 ns, total: 5.73 ms
Wall time: 5.72 ms
CPU times: user 854 µs, sys: 0 ns, total: 854 µs
Wall time: 855 µs


In [17]:
#print(len([word for word in texts[0] if word not in stopwords]))
print(len(texts[0]))

5


Below are some Gensim specific conversions; I also filter out extreme words (see inline comment)

In [18]:
#create a Gensim dictionary from the texts
dictionary = corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in texts]

In [19]:
len(corpus)

8

The actual model runs below. I took 1825 passes to ensure convergence, but you can see that it took my machine 4 minutes to run. My chunksize is larger than the corpus so basically all synopses are used per pass. I should optimize this, and Gensim has the capacity to run in parallel. I'll likely explore this further as I use the implementation on larger corpora.

In [20]:
# %time lda = models.LdaModel(corpus, num_topics=10, id2word=dictionary, update_every=5, chunksize=10000, passes=100)
%time lda = models.LdaModel(corpus, num_topics=5, id2word=dictionary, update_every=5, chunksize=10000, passes=100)

CPU times: user 331 ms, sys: 4.02 ms, total: 335 ms
Wall time: 342 ms


Each topic has a set of words that defines it, along with a certain probability.

In [21]:
# lda.show_topics()

pprint(lda.print_topics())

[(0,
  '0.052*"generat" + 0.039*"file" + 0.039*"string" + 0.027*"docstr" + '
  '0.027*"first" + 0.027*"loadtxt" + 0.027*"yield" + 0.027*"filenam" + '
  '0.015*"argument" + 0.015*"read"'),
 (1,
  '0.035*"improv" + 0.019*"propos" + 0.019*"renam" + 0.019*"notebook" + '
  '0.019*"perfect" + 0.019*"play" + 0.019*"needl" + 0.019*"rememb" + '
  '0.019*"shift-tab" + 0.019*"searchsort"'),
 (2,
  '0.076*"shape" + 0.043*"int" + 0.035*"mismatch" + 0.026*"array" + '
  '0.026*"exampl" + 0.026*"valu" + 0.026*"error" + 0.018*"could" + '
  '0.018*"messag" + 0.018*"result"'),
 (3,
  '0.056*"benchmark" + 0.056*"test" + 0.034*"function" + 0.024*"becaus" + '
  '0.024*"time" + 0.024*"singl" + 0.024*"suit" + 0.024*"lie" + 0.024*"run" + '
  '0.024*"multipli"'),
 (4,
  '0.065*"rais" + 0.045*"return" + 0.024*"user" + 0.024*"sure" + '
  '0.024*"assign" + 0.024*"except" + 0.024*"ticket" + 0.024*"anyth" + '
  '0.024*"rational" + 0.024*"nan"')]


In [22]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.185152 -0.041427       1        1  35.779821
3     -0.120866 -0.118747       2        1  24.837407
0     -0.055419  0.117424       3        1  19.980616
1     -0.002906  0.026209       4        1  12.362445
4     -0.005961  0.016541       5        1   7.039712, topic_info=          Term      Freq     Total Category  logprob  loglift
119      shape  7.000000  7.000000  Default  30.0000  30.0000
158       rais  1.000000  1.000000  Default  29.0000  29.0000
64   benchmark  3.000000  3.000000  Default  28.0000  28.0000
93        test  3.000000  3.000000  Default  27.0000  27.0000
135    generat  2.000000  2.000000  Default  26.0000  26.0000
..         ...       ...       ...      ...      ...      ...
61      becaus  0.070397  2.484906   Topic5  -5.5094  -0.9102
94        time  0.070397  2.484906   Topic5  -5.5094  -0.9102
31         bit  0.070397  1.292911   Topic5  -5.5094  -0.2569
33     correct  0.070397  1.292911   Topic5  -5.5094  -0.2569
37        give  0.070397  1.292911   Topic5  -5.5094  -0.2569

[204 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
121       3  0.853801         .bz2
122       3  0.853801          .gz
100       1  0.773448      .reshap
123       3  0.853801           3k
101       1  0.773448  __setitem__
...     ...       ...          ...
120       1  0.727570         valu
57        1  0.989661         warn
148       3  0.853801         well
149       3  0.853801          yet
150       3  0.562579        yield

[144 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 1, 2, 5])

In [23]:
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)#0.5588166880236373


Coherence Score:  0.7486601758791229


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4121e9a6-526b-4eeb-ae8d-65c5504611ef' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>